# 1) Overlap Between Maps

For a new redistricting map, show what old districts overlap with each new district.

Make sure rtree is installed in your local environment. See this notebook for your previous struggles: https://github.com/devinbrady/dc-parking/blob/main/Spatial-Join.ipynb

In [1]:
import pandas as pd
import geopandas as gp
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Use this projection for measuring distances in meters
# https://octo.dc.gov/page/coordinate-system-standards
dc_crs = ('epsg', '26985')

# Use WGS84 for saving GeoJSON for display in QGIS
output_crs = ('epsg', '4326')

In [3]:
# Read in data
previous_map = gp.read_file('../uploads/to-mapbox-smd-data.geojson')
new_map = gp.read_file('Single_Member_District_from_2023.geojson')

In [4]:
# Set column names on the new map
new_map['smd_name'] = new_map['SMD_ID']
new_map['smd_id'] = 'smd_2022_' + new_map['SMD_ID']
new_map.rename(columns={'ANC_ID': 'anc_id'}, inplace=True)

# We only need these columns going forward
map_columns = ['smd_id', 'smd_name', 'anc_id', 'geometry']

# Save out the new map for later steps
new_map[map_columns].to_file('to-mapbox-2022-smd-data-overlap.geojson', driver='GeoJSON')

In [5]:
# Convert to local coordinate reference system
previous_map = previous_map.to_crs(dc_crs)
new_map = new_map.to_crs(dc_crs)

In [6]:
previous_map['district_area'] = previous_map.geometry.area
new_map['district_area'] = new_map.geometry.area

In [7]:
def calculate_overlap(map_a, map_b, suffix_a, suffix_b):

    overlap = map_a[overlap_columns].overlay(
        map_b[overlap_columns], how='identity', keep_geom_type=False
    )
    
    overlap.rename(columns={
        'smd_id_1': 'smd_id' + suffix_a
        , 'smd_id_2': 'smd_id' + suffix_b
        , 'district_area_1': 'district_area' + suffix_a
        , 'district_area_2': 'district_area' + suffix_b
    }, inplace=True)

    overlap['overlap_area'] = overlap.geometry.area
    overlap['overlap_perc'] = overlap['overlap_area'] / overlap['district_area' + suffix_a]
    
    overlap['district_rank'] = overlap.groupby('smd_id' + suffix_a).overlap_perc.rank(ascending=False)
    overlap.sort_values(by=['smd_id' + suffix_a, 'overlap_area'], ascending=[True, False], inplace=True)
    
    overlap.drop('geometry', axis=1, inplace=True)
    
    return overlap

In [8]:
# Calculate the overlap
overlap_columns = ['smd_id', 'district_area', 'geometry']

In [9]:
# Backwards - how much of each new district is made up of the old districts? 
# Use this for the frontend list for each new district. 
overlap_backwards = calculate_overlap(new_map[overlap_columns], previous_map[overlap_columns], '_2022', '_2012')

In [10]:
# Forwards - how much of each old district went into this new district? 
# Use this for colors for continuity
overlap_forwards = calculate_overlap(previous_map[overlap_columns], new_map[overlap_columns], '_2012', '_2022')

In [11]:
# Save results
columns_for_csv = [c for c in overlap_backwards.columns if c != 'geometry']
overlap_backwards[columns_for_csv].to_csv('overlap_backwards.csv', index=False)
overlap_forwards[columns_for_csv].to_csv('overlap_forwards.csv', index=False)

In [12]:
# Should be 345
(overlap_backwards.district_rank == 1).sum()

345

In [13]:
# Should be 296
(overlap_forwards.district_rank == 1).sum()

296

## Examples

Every old district that is in this new district:

In [14]:
overlap_forwards[overlap_forwards.smd_id_2022 == 'smd_1D07']

,smd_id_2012,district_area_2012,smd_id_2022,district_area_2022,overlap_area,overlap_perc,district_rank


Every new district that is made up of parts of this old district:

In [15]:
overlap_forwards[overlap_forwards.smd_id_2012 == 'smd_1C07']

,smd_id_2012,district_area_2012,smd_id_2022,district_area_2022,overlap_area,overlap_perc,district_rank
286,smd_1C07,203187.80,smd_2022_1C07,129160.04,111510.19,0.55,1.00
267,smd_1C07,203187.80,smd_2022_1C08,128342.79,36327.84,0.18,2.00
348,smd_1C07,203187.80,smd_2022_1C05,112337.86,34708.44,0.17,3.00
261,smd_1C07,203187.80,smd_2022_1B07,95592.43,12838.70,0.06,4.00
300,smd_1C07,203187.80,smd_2022_1C01,135254.01,7669.77,0.04,5.00
335,smd_1C07,203187.80,smd_2022_1C04,330058.75,132.81,0.00,6.00
306,smd_1C07,203187.80,smd_2022_2B08,111970.17,0.02,0.00,7.00
343,smd_1C07,203187.80,smd_2022_1C06,82575.03,0.02,0.00,8.00
168,smd_1C07,203187.80,smd_2022_1C09,90633.10,0.01,0.00,9.00
292,smd_1C07,203187.80,smd_2022_1C03,119796.17,0.00,0.00,10.00


Every new district that this old district went to make: 

In [16]:
overlap_backwards[overlap_backwards.smd_id_2022 == 'smd_1C08']

,smd_id_2022,district_area_2022,smd_id_2012,district_area_2012,overlap_area,overlap_perc,district_rank


In [17]:
overlap_forwards[overlap_forwards.smd_id_2012 == 'smd_1B08']

,smd_id_2012,district_area_2012,smd_id_2022,district_area_2022,overlap_area,overlap_perc,district_rank
162,smd_1B08,96186.90,smd_2022_1A08,80997.35,41104.32,0.43,1.00
237,smd_1B08,96186.90,smd_2022_1B09,108656.14,30462.69,0.32,2.00
60,smd_1B08,96186.90,smd_2022_1A09,127755.47,24619.90,0.26,3.00
217,smd_1B08,96186.90,smd_2022_1B03,219595.07,0.00,0.00,4.00
251,smd_1B08,96186.90,smd_2022_1B06,132533.63,0.00,0.00,5.00
1982,smd_1B08,96186.90,NaN,nan,0.00,0.00,6.00
87,smd_1B08,96186.90,smd_2022_1A01,85597.19,0.00,0.00,7.00
115,smd_1B08,96186.90,smd_2022_1A10,163553.17,0.00,0.00,8.00
